In [1]:
import jieba
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [2]:
from os import listdir
from os.path import isfile, join,splitext
t_dir = 'dataTrainComplete'
txt_fnames = [splitext(f)[0] for f in listdir(t_dir) if isfile(join(t_dir, f))]

In [3]:
crop_list = open('Keywords/02crop.list.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()

pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()

In [4]:
from itertools import chain
vector_dict = {}
for idx,line in enumerate(chain(crop_line_sep,pest_line_sep,chem_line_sep)):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[word] = idx

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\NTHU-\AppData\Local\Temp\jieba.cache
Loading model cost 0.881 seconds.
Prefix dict has been built successfully.


In [5]:
vectors = {}
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    vectors[fname] = [0]*764
    for seg in seg_list:
        if(seg in vector_dict):
            vectors[fname][vector_dict[seg]] = 1

In [7]:
labels = {}
pair_list = []
for fname1 in txt_fnames:
    for fname2 in txt_fnames:
        if(fname1 == fname2):continue
        else:
            labels[(fname1,fname2)] = 0
            pair_list.append((fname1,fname2))
corr_list = open('TrainLabel.csv', "r",encoding='UTF-8-sig')
corr = corr_list.read()
corr_line_sep = corr.splitlines()
for line in corr_line_sep[1:]:
    l = line.split(',')
    labels[(l[0],l[1])] = 1
print(len(labels))
print(sum(labels.values()))
print(len(pair_list))

313040
1383
313040


In [29]:
labels.keys()

dict_keys([('1', '10'), ('1', '1000'), ('1', '1005'), ('1', '1007'), ('1', '1010'), ('1', '1011'), ('1', '1015'), ('1', '1016'), ('1', '1023'), ('1', '1025'), ('1', '103'), ('1', '1033'), ('1', '1037'), ('1', '1039'), ('1', '104'), ('1', '1041'), ('1', '1049'), ('1', '105'), ('1', '1052'), ('1', '1054'), ('1', '1056'), ('1', '1066'), ('1', '1067'), ('1', '1068'), ('1', '107'), ('1', '1073'), ('1', '1074'), ('1', '1075'), ('1', '1076'), ('1', '108'), ('1', '1082'), ('1', '1086'), ('1', '1087'), ('1', '1088'), ('1', '1093'), ('1', '110'), ('1', '1103'), ('1', '1111'), ('1', '1114'), ('1', '1116'), ('1', '1118'), ('1', '1119'), ('1', '1120'), ('1', '1125'), ('1', '1126'), ('1', '1127'), ('1', '1128'), ('1', '113'), ('1', '1130'), ('1', '1132'), ('1', '1133'), ('1', '1141'), ('1', '1142'), ('1', '1146'), ('1', '1147'), ('1', '1148'), ('1', '1151'), ('1', '1153'), ('1', '1155'), ('1', '1157'), ('1', '1159'), ('1', '1161'), ('1', '1165'), ('1', '1166'), ('1', '1167'), ('1', '117'), ('1', '11

In [33]:
X_train, X_val, y_train, y_val = train_test_split(list(labels.keys()), list(labels.values()), test_size=0.2, random_state=42, stratify= list(labels.values()))

In [34]:
X_train

[('25', '753'),
 ('403', '694'),
 ('979', '25'),
 ('46', '1199'),
 ('695', '821'),
 ('992', '215'),
 ('113', '1249'),
 ('35', '893'),
 ('1128', '10'),
 ('298', '236'),
 ('1119', '371'),
 ('586', '1339'),
 ('700', '856'),
 ('747', '237'),
 ('474', '1337'),
 ('1073', '582'),
 ('313', '1256'),
 ('1300', '365'),
 ('1116', '37'),
 ('692', '171'),
 ('692', '583'),
 ('1157', '1125'),
 ('197', '178'),
 ('1172', '725'),
 ('1334', '176'),
 ('508', '631'),
 ('949', '1357'),
 ('730', '1350'),
 ('1005', '108'),
 ('1301', '553'),
 ('747', '1263'),
 ('282', '300'),
 ('694', '587'),
 ('1341', '104'),
 ('713', '992'),
 ('755', '522'),
 ('186', '774'),
 ('1319', '286'),
 ('845', '1305'),
 ('1041', '1339'),
 ('248', '1067'),
 ('119', '1364'),
 ('1298', '299'),
 ('663', '361'),
 ('1037', '725'),
 ('870', '1401'),
 ('475', '1319'),
 ('709', '672'),
 ('1401', '306'),
 ('267', '140'),
 ('1088', '40'),
 ('351', '544'),
 ('791', '86'),
 ('1271', '716'),
 ('916', '806'),
 ('313', '150'),
 ('847', '716'),
 ('988

In [8]:
# Dataset
class PartDataset(data.Dataset):
    
    def __init__(self, labels, pair_list, vectors, phase='train'):
        self.labels = labels
        self.pair_list = pair_list
        self.vectors = vectors
        self.phase = phase
        
    def __len__(self):
        return len(self.pair_list)
    
    def __getitem__(self, idx):
        Test,Ref = pair_list[idx]
        label = self.labels[(Test,Ref)]
        comb_vector = self.vectors[Test] + self.vectors[Ref]
        return torch.tensor(comb_vector), label

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
train_dataset = PartDataset(labels, txt_fnames, vectors,phase='train')
test_dataset = PartDataset(labels, txt_fnames, vectors,phase='test')

In [11]:
print('Operation Check')
print(labels[('3','415')])
print(pair_list.index(('3','415')))
print(train_dataset.__getitem__(133886)[1])

Operation Check
1
133886
1


In [12]:
# DataLoader
train_dataloader = data.DataLoader(train_dataset, batch_size=4000, shuffle=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=4000, shuffle=False)

dataloader_dict = {'train': train_dataloader, 'test': test_dataloader}

# Operation Check
print('Operation Check')
batch_iterator = iter(train_dataloader)
inputs, label = next(batch_iterator)
print(len(label))

Operation Check
560


In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2*764, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        output = torch.sigmoid(logits)
        #return logits        
        return output

In [18]:
net = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=net.parameters(), lr=0.001, momentum=0.9)

In [19]:
import time
def train_model(net, dataloader_dict, criterion, optimizer, num_epoch):
    
    since = time.time()
    best_model_wts = copy.deepcopy(net.state_dict())
    best_acc = 0.0
    net = net.to(device)
    
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)
        
        for phase in ['train', 'test']:
            
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            for inputs, labels in tqdm(dataloader_dict[phase]):
                inputs = inputs.type(torch.FloatTensor).to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(net.state_dict())
                print(time.time())
                torch.save(net.state_dict(), 'best_checkpoint_last.pth')
                
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    net.load_state_dict(best_model_wts)
    return net

In [20]:
num_epoch = 100
net = train_model(net, dataloader_dict, criterion, optimizer, num_epoch)

Epoch 1/100
--------------------


  0%|          | 0/1 [00:00<?, ?it/s]

train Loss: 0.6938 Acc: 0.0786


  0%|          | 0/1 [00:00<?, ?it/s]

test Loss: 0.6938 Acc: 0.0893
1638791432.7998662
Epoch 2/100
--------------------


...

test Loss: 0.6543 Acc: 1.0000
Epoch 100/100
--------------------


  0%|          | 0/1 [00:00<?, ?it/s]

train Loss: 0.6543 Acc: 1.0000


  0%|          | 0/1 [00:00<?, ?it/s]

test Loss: 0.6539 Acc: 1.0000
Training complete in 0m 32s
Best val Acc: 1.000000
